In [1]:
import pandas as pd
import numpy as np

### 1. Постановка задачи

Необходимо написать программу, используя не эзотерический язык программирования (например, Python, Ruby, C/C++, R, …), которая читает предоставленный файл и выдаёт таблицу с аналитикой по шаблону ниже, в виде текстового документа (.txt, MS Word или PDF).

Формат вывода:

![picture](picture.png) 

### 2. Понимание данных

Формат файла: SUBJID, VISITN, VISIT, PARAMCD, PARAM, AVAL, TRTGRPN, ITTFL, PPFL; разделитель — точка с запятой ";"

Пояснения к формату:
* SUBJID — Subject unique number
* VISITN — Visit sequence number
* VISIT — Visit name
* PARAMCD — Parameter code
* PARAM — Parameter name
* AVAL — Parameter result
* TRTGRPN — Treatment group number
* ITTFL — Intention to treat population flag
* PPFL — Per protocol pupulation flag

В данных обнаружено одно пропущенное значение и один незапланированный визит

Также тип числовых данных был некорректно прочитан, что потребует внесения корректировок

In [2]:
# считываем данные
df = pd.read_csv('biocad-sas.csv', ';')

# изучаем структуру датасета
df.head(10)

,SUBJID,VISITN,VISIT,PARAMCD,PARAM,AVAL,TRTGRPN,ITTFL,PPFL
0,Subject unique number,Visit sequence number,Visit name,Parameter code,Parameter name,Parameter result,Treatment group number,Intention to treat population flag,Per protocol pupulation flag
1,111,1,Visit 1,EFF01,Efficacy Parameter 1,15.3,1,1,1
2,111,2,Visit 2,EFF01,Efficacy Parameter 1,75,1,1,1
3,111,3,Visit 3,EFF01,Efficacy Parameter 1,62,1,1,1
4,111,1,Visit 1,EFF02,Efficacy Parameter 2,97.89,1,1,1
5,111,3,Visit 3,EFF02,Efficacy Parameter 2,23.1,1,1,1
6,112,1,Visit 1,EFF01,Efficacy Parameter 1,17.4,2,1,1
7,112,2,Visit 2,EFF01,Efficacy Parameter 1,84.2,2,1,1
8,112,3,Visit 3,EFF01,Efficacy Parameter 1,5.4,2,1,1
9,112,1,Visit 1,EFF02,Efficacy Parameter 2,82.33,2,1,1


In [3]:
# исследуем информацию о типах данных, проверяем на пропущенные значения
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   SUBJID   54 non-null     object
 1   VISITN   54 non-null     object
 2   VISIT    54 non-null     object
 3   PARAMCD  54 non-null     object
 4   PARAM    54 non-null     object
 5   AVAL     53 non-null     object
 6   TRTGRPN  54 non-null     object
 7   ITTFL    54 non-null     object
 8   PPFL     54 non-null     object
dtypes: object(9)
memory usage: 3.9+ KB


In [4]:
# проверяем распеределние значений визитов
df.VISIT.value_counts()

Visit 1        22
Visit 3        20
Visit 2        10
Visit name      1
Unscheduled     1
Name: VISIT, dtype: int64

### 3. Подготовка данных

Для проведения анализа исключим из датасета первую строку, которая поясняет названия столбцов и значения столбцов с названием визита и параметра. Удалим пропущенное значение и информацию о незапланированном визите

In [5]:
# удаляем информацию о незапланированном визите
data = df[df['VISIT'] != 'Unscheduled']

# исключаем первую строку, значения столбцов VISIT, PARAM и удаляем пропущенное значение
data = data[['SUBJID', 'VISITN', 'PARAMCD', 'AVAL', 'TRTGRPN', 'ITTFL', 'PPFL']][1:].dropna(axis = 0, how ='any')

In [6]:
data.head(10)

,SUBJID,VISITN,PARAMCD,AVAL,TRTGRPN,ITTFL,PPFL
1,111,1,EFF01,15.3,1,1,1
2,111,2,EFF01,75,1,1,1
3,111,3,EFF01,62,1,1,1
4,111,1,EFF02,97.89,1,1,1
5,111,3,EFF02,23.1,1,1,1
6,112,1,EFF01,17.4,2,1,1
7,112,2,EFF01,84.2,2,1,1
8,112,3,EFF01,5.4,2,1,1
9,112,1,EFF02,82.33,2,1,1
10,112,3,EFF02,54.34,2,1,1


Приведем данные в наборе к нужному типу:
* SUBJID, VISITN, TRTGRPN, PPFL — целое число
* PARAMCD — строка
* AVAL — число с плавающей запятой

In [7]:
# проходим циклом по названиям столбцов, тип которых нужно изменить
for i in ['SUBJID', 'VISITN', 'AVAL', 'TRTGRPN', 'ITTFL', 'PPFL']:
    
    # указываем тип float для данных в столбце AVAL, для остальных — int
    if i != 'AVAL':
        data = data.astype({i: np.int})
    else: data = data.astype({i: np.float})

In [8]:
# проводим проверку проведенных преобразований
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 1 to 53
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SUBJID   51 non-null     int32  
 1   VISITN   51 non-null     int32  
 2   PARAMCD  51 non-null     object 
 3   AVAL     51 non-null     float64
 4   TRTGRPN  51 non-null     int32  
 5   ITTFL    51 non-null     int32  
 6   PPFL     51 non-null     int32  
dtypes: float64(1), int32(5), object(1)
memory usage: 2.2+ KB


В результате получаем датасет, пригодный для дальнейшего проведения анализа

### 4. Анализ данных

Напишем программу, которая получает на вход данные для анализа, номер параметра эффективности, группу ITTFL или PPFL, и возвращает txt файл с аналитикой в рамках предложенного шаблона 

In [9]:
# создаем функцию для расчета необходимых показателей, чтобы использовать ее далее
def stats(d, trtgrpn, visitn):
    return d[(d['TRTGRPN'] == trtgrpn) & (d['VISITN'] == visitn)]['AVAL'].describe().drop(['25%', '50%', '75%'])

In [10]:
# создаем функцию, которая записывает в текстовый файл результаты расчетов по заданым характеристикам
def data_analysis(d, parmacd, ittfl_or_ppfl, i=1):
   
    # фильтруем данные по параметру эффективности и ittfl/ppfl
    df1 = d[(d['PARAMCD'] == parmacd) & (d[ittfl_or_ppfl] == i)]
   
    # создаем текстовый файл, куда будем записывать таблицу с результатами проведенного анализа данных
    with open('statistics.txt', 'w') as f:
        
        # выводим название параметра и тип группы исследования в зависимости от переданных переменных
        if parmacd == 'EFF01':
            if ittfl_or_ppfl == 'ITTFL':
                print('Table: Summary of Efficacy Parameter 1 by Visit. Intention-to-Treat population.', file=f)
            else: 
                print('Table: Summary of Efficacy Parameter 1 by Visit. Per-protocol-pupulation flag.', file=f)
        else:
            if ittfl_or_ppfl == 'ITTFL':
                print('Table: Summary of Efficacy Parameter 2 by Visit. Intention-to-Treat population.', file=f)
            else:
                print('Table: Summary of Efficacy Parameter 2 by Visit. Per-protocol-pupulation flag.', file=f)
        
        # выводим информацию о количестве пациентов в каждом номере группы лечения
        print(f'Visit                           Treatment group 1 (N={df1[df1.TRTGRPN == 1].shape[0]})              Treatment group 2 (N={df1[df1.TRTGRPN == 2].shape[0]})', file=f)
        print('  Statistics', file=f) 
        
        # с помощью вложенной функции расчитываем показатели и выводим их для каждой группы лечения по визитам 
        for vn in range(1,4):
            if round(stats(df1, 1, vn)[0]) != 0:
                print('Visit', vn, file=f)
                print(f'  n                             {round(stats(df1, 1, vn)[0])}                                     {round(stats(df1, 2, vn)[0])}', file=f)
                print(f'  Mean                          {round(stats(df1, 1, vn)[1], 1)}                                  {round(stats(df1, 2, vn)[1], 1)}', file=f)
                print(f'  Standard Deviation            {round(stats(df1, 1, vn)[2], 2)}                                 {round(stats(df1, 2, vn)[2], 2)}', file=f)
                print(f'  Minimum                       {round(stats(df1, 1, vn)[3])}                                    {round(stats(df1, 2, vn)[3])}', file=f)
                print(f'  Maximum                       {round(stats(df1, 1, vn)[4])}                                    {round(stats(df1, 2, vn)[4])}', file=f)
            else: continue
        print('\nN: Number of subjects in the population and treatment group.', file=f)

### Результат

Расчитываем необходимые статистические метрики и построим итоговую таблицу для параметра эффективности №1 и группы Intention-to-Treat population, опираясь на предложенный шаблон вывода в задаче 

In [12]:
# вызываем функцию для параметров эффективности 1 с Intention-to-Treat population
data_analysis(data, 'EFF01', 'ITTFL')

# проверяем полученный результат
with open('statistics.txt', 'r') as f:
    print(f.read())

Table: Summary of Efficacy Parameter 1 by Visit. Intention-to-Treat population.
Visit                           Treatment group 1 (N=16)              Treatment group 2 (N=11)
  Statistics
Visit 1
  n                             6                                     3
  Mean                          43.0                                  28.4
  Standard Deviation            29.33                                 31.0
  Minimum                       15                                    4
  Maximum                       90                                    63
Visit 2
  n                             5                                     4
  Mean                          35.1                                  64.7
  Standard Deviation            27.12                                 18.86
  Minimum                       10                                    47
  Maximum                       75                                    84
Visit 3
  n                             5                   